In [ ]:
import geopandas as gpd
import pandas as pd
import plotly.express as px
import json

In [ ]:
df_map = gpd.read_file('bgr_admbnda_adm1_UNICEF_2019.shp')
geo_json = json.loads(df_map[['ADM1_EN', 'geometry']].to_json())
df_salary_old = pd.read_csv('salary2000.csv', delimiter = ';')
df_salary_new = pd.read_csv('salary2008.csv', delimiter = ';')

In [ ]:
#Delete extra column
df_salary_old = df_salary_old.iloc[:, :-1]
#Change headers
df_salary_old.drop(df_salary_old.index[0:2], inplace=True)
df_salary_old.reset_index(drop=True, inplace=True)
df_salary_old.columns = ['City', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007']

In [ ]:
#Delete extra column
df_salary_new = df_salary_new.iloc[:, :-1]
#Change headers
df_salary_new.drop(df_salary_new.index[0:2], inplace=True)
df_salary_new.reset_index(drop=True, inplace =True)
df_salary_new.columns = ['City', '2008', '2009', '2010', '2011', '2012', '2013', '2014', 
                         '2015', '2016', '2017', '2018', '2019']

In [ ]:
df_whole = pd.merge(df_salary_old, df_salary_new)

#Fix naming
df_whole['City'][df_whole['City'] == 'София'] = 'Софийска'
df_whole['City'][df_whole['City'] == 'София (столица)'] = 'София-град'

In [ ]:
df = pd.merge(df_whole, df_map[['ADM1_BG','ADM1_EN']], left_on='City', right_on='ADM1_BG')

In [ ]:
df.drop(columns=['ADM1_BG', 'City'], inplace=True)

In [ ]:
cols_num = df.columns.drop(['ADM1_EN'])

In [ ]:
df[cols_num] = df[cols_num].apply(pd.to_numeric, errors='coerce')

In [ ]:
df_melted = df.melt(id_vars='ADM1_EN')
df_final = df_melted.rename(columns = {'variable': 'Year', 'value': 'Salary'})

In [ ]:
fig = px.choropleth(df_final, geojson=geo_json, color='Salary',
                           color_continuous_scale='plasma',
                           range_color=(2000, 22000),
                           locations='ADM1_EN',
                           featureidkey='properties.ADM1_EN',
                           animation_frame='Year'
                          )

fig.update_layout(title_text='Average annual salary in Bulgarian provinces in leva', title_x=0.5)
fig.update_geos(fitbounds="locations", visible=False)
fig.show()